# Primeiro time a marcar um gol

Este notebook tem como principal objetivo utilizar o algorito K-Means para determinar qual time possui a maior probabilidade de marcar o primeiro gol em uma partida.

### K-Means - Algoritmo utilizado no modelo

O K-Means é um algoritmo de aprendizado não supervisionado que agrupa dados em "K" clusters com base em suas características. Ele funciona atribuindo cada ponto de dado ao cluster mais próximo (com base na distância euclidiana), recalculando as médias dos clusters e repetindo até que as atribuições não mudem mais.

In [17]:
# Import das bibliotecas que serão utilizadas no modelo preditivo
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix, accuracy_score, precision_recall_curve, silhouette_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import label_binarize

Leitura dos dados e merge dos dados de times, transformando-os em um único dataframe.

In [18]:
matches = pd.read_csv('../../data/brazil-serie-a-matches-2024-to-2024-stats (5).csv')
times1 = pd.read_csv('../../data/brazil-serie-a-teams-2024-to-2024-stats (4).csv')
times2 = pd.read_csv('../../data/brazil-serie-a-teams2-2024-to-2024-stats (2).csv')

# Merge dos dataframes times1 e times2, transformando-os em um único dataframe teams
teams = pd.merge(times1, times2, on="common_name", how="left")

# Cria uma coluna chamada team_index, visando criar um indíce numérico para cada time
teams.insert(0, "team_index", list(range(1, 21)))

## Limpeza de dados

Limpeza de dados do dataframe de partidas, visando reduzir o dataframe apenas a tabelas importantes, além de remover dados vazios e partidas suspensas, canceladas e incompletas.

Além disso, o SimpleImputer é utilizado para substituir valorez vazios pela média dos valores da coluna, estratégia que se provou mais eficaz do que substituir pela mediana ou por desconsiderar a linha.

In [19]:
categorical_matches = pd.DataFrame(matches)
categorical_matches = categorical_matches[~categorical_matches['status'].isin(['suspended', 'canceled', 'incomplete'])]
#categorical_matches = categorical_matches[~categorical_matches['first_to_score'].isin(['N/A'])]

matches = matches[["status", "home_team_name", "away_team_name", "first_to_score"]]
matches = matches[~matches['status'].isin(['suspended', 'canceled', 'incomplete'])]
matches = matches[~matches['first_to_score'].isin(['N/A'])]

In [20]:
# Separação do dataframe teams em um dataframe com colunas categóricas e um dataframe com colunas numéricas
teams_categorical = teams
teams = teams.select_dtypes(include=['int64', 'float64'])

# Limpeza com imputer
imputer_numbers = SimpleImputer(missing_values=np.nan, strategy="median")
imputer_numbers.fit(teams)

Seleção de features e criação do dataframe que será utilizado para treinar o modelo preditivo.

In [22]:
selected_tables = [
    "wins",
    "draws",
    "losses",
    "first_team_to_score_count",
    "goals_scored",
    "goals_conceded",
    "clean_sheets",
    "btts_count",
    "fts_count",
]

train_data = pd.DataFrame(teams[selected_tables])

selected_tables.insert(1, "common_name")

data = pd.DataFrame(teams)
data["common_name"] = teams_categorical["common_name"]

## Definição de hiperparâmetros

Utilizando Grid Search, são testadas diversas configurações para o K-Means, visando encontrar a melhor configuração do modelo. Para chegar ao resultado do melhor modelo, é utiliza o índice da silhueta.

In [ ]:
def silhouette_scorer(estimator, data):
    labels = estimator.fit_predict(data)
    return silhouette_score(data, labels)

kmeans = KMeans()

param_grid = {
    'n_clusters': [3],
    'n_init': [10, 20, 30],
    'max_iter': [300, 500],
    'tol': [1e-4, 1e-3]
}

grid_search = GridSearchCV(estimator=kmeans, param_grid=param_grid, scoring=silhouette_scorer, cv=3)
grid_search.fit(teams)

## Definição do algoritmo

Definição do algoritmo K-Means, passando as melhores configurações definidas pelos hiperparâmetros.

In [ ]:
num_clusters = grid_search.best_params_['n_clusters']
max_iter = grid_search.best_params_['max_iter']
n_init = grid_search.best_params_['n_init']
tol = grid_search.best_params_['tol']
kmeans = KMeans(n_clusters=num_clusters, max_iter=max_iter, n_init=n_init, tol=tol)
kmeans.fit(train_data)

In [25]:
centroids = kmeans.cluster_centers_
labels = kmeans.labels_

Plotagem de um gráfico de barras, visando demonstrar os pesos de cada feature nas previsões do modelo.

In [ ]:
variable_weights = pd.DataFrame(dict(zip(train_data.columns, kmeans.cluster_centers_[labels])))
variable_weights = variable_weights.mean().sort_values()

plt.figure(figsize=(10, 6))
plt.barh(variable_weights.index, variable_weights.values, color='skyblue')
plt.xlabel("Média dos Pesos")
plt.title("Variáveis Ordenadas pelos Maiores Pesos")
plt.show()

matches = pd.DataFrame(matches)

## Plotagem dos gráficos dos clusters gerados pelo K-Means

O K-Means teve a responsabilidade de separar cada time participante do Brasilerão em três categorias, sendo elas, times pequenos, times médios e times grandes. A partir dessa separação é possível calcular a probabilidade de cada time fazer o primeiro gol em uma partida.

In [ ]:
plt.scatter(data[labels == 2]['first_team_to_score_count'], data[labels == 2]['clean_sheets'], color='red', label='Grandes')
plt.scatter(data[labels == 1]['first_team_to_score_count'], data[labels == 1]['clean_sheets'], color='blue', label='Médio')
plt.scatter(data[labels == 0]['first_team_to_score_count'], data[labels == 0]['clean_sheets'], color='green', label='Pequeno')
plt.scatter(centroids[:, 0], centroids[:, 1], color='black', marker='x', label='Centroids')
plt.xlabel('Wins')
plt.ylabel('Losses')
plt.title('Clusters dos times')
plt.legend()
plt.show()

fig = px.scatter_3d(train_data, x='first_team_to_score_count', y='fts_count', z='clean_sheets', color='first_team_to_score_count', title='Clusters dos times', labels={
        'first_team_to_score_count': 'Primeiro Time a Marcar',
        'fts_count': 'FTS',
        'clean_sheets': 'Jogos Sem Sofrer Gol'
    })
fig.add_scatter3d(x=centroids[:, 0], y=centroids[:, 1], z=centroids[:, 2], mode='markers', marker=dict(color='black', size=2, symbol='x'), name='Centroids')
fig.show()

Criação do DataFrame de resultados do modelo, e os times, que antes estavam separados entre os clusters 0, 1 e 2, agora foram separados de forma categórica, indicando os grupos através dos títulos Pequeno, Médio e Grande.

In [28]:
results_km = pd.DataFrame()
results_km["team_index"] = data["team_index"]
results_km["team_name"] = data["common_name"]
results_km["labels_classif"] = labels

classificacao = list()
for i in range(0, 20):
    mapeamento_classificacao = {0: "Grande", 1: "Médio", 2: "Pequeno"}
    classificacao.append(mapeamento_classificacao.get(results_km["labels_classif"][i], "Desconhecido"))

results_km["classificacao"] = classificacao

In [29]:
times_classificados = { "Pequeno": list(), "Médio": list(), "Grande": list() }

index = 0
for key, value in results_km["classificacao"].items():
    times_classificados[value].append(results_km["team_name"][index])
    index += 1

## Cálculo da probabilidade

Para calcular a probabilidade de cada time fazer o primeiro gol, foi utilizada a seguinte fórmula matemática:

P(1° gol) = TAxX / Qpm

- **TAxX**: O TAxX é o total de 1°s gols do time A contra times do grupo X, sendo o grupo X o grupo ao qual o time B pertence.
- **Qpm**: Quantidade de partidas que o time A disputou contra times do grupo X, sendo o grupo X o grupo ao qual o time B pertence.

In [30]:
def calcular_probabilidade(time_a, time_b):
    grupo_time_b = list()
    for grupo in times_classificados.values():
        if grupo.__contains__(time_b):
            grupo_time_b = grupo

    a_team_matches = matches.query(f"home_team_name == '{time_a}' or away_team_name == '{time_a}'")
    a_team_matches_against_group_b = a_team_matches[a_team_matches['away_team_name'].isin(grupo_time_b) | a_team_matches['home_team_name'].isin(grupo_time_b)]

    TAxX = a_team_matches_against_group_b[a_team_matches_against_group_b['first_to_score'] == time_a].shape[0]

    Qpm = len(a_team_matches_against_group_b)

    probabilidade = TAxX / Qpm if TAxX > 0 and Qpm > 0 else 0.5

    return probabilidade

Considerando que a função `calcular_probabilidade()` retorna a probabilidade de cada time fazer o primeiro gol, a função `comparar_times()` retorna o nome que tem a maior probabilidade de fazer o primeiro gol.

In [31]:
def comparar_times(time_a, time_b):
    prob_a = calcular_probabilidade(time_a, time_b)
    prob_b = calcular_probabilidade(time_b, time_a)

    soma_probabs = prob_a + prob_b
    prob_a = (prob_a / soma_probabs) * 100
    prob_b = (prob_b / soma_probabs) * 100
    
    if prob_a == prob_b:
        return "Empate"
    
    return time_a if prob_a > prob_b else time_b

In [ ]:
def prever_primeiro_gol(row):
    time_a = row['home_team_name']
    time_b = row['away_team_name']
    resultado = comparar_times(time_a, time_b)

    if time_a in resultado:
        return time_a
    elif time_b in resultado:
        return time_b
    else:
        return 'Empate'

matches['previsao_primeiro_gol'] = matches.apply(prever_primeiro_gol, axis=1)

matches = matches.dropna(subset=['previsao_primeiro_gol', 'first_to_score'])

matches['previsao_primeiro_gol'] = matches['previsao_primeiro_gol'].astype(str)
matches['first_to_score'] = matches['first_to_score'].astype(str)

y_true = matches['first_to_score']
y_pred = matches['previsao_primeiro_gol']

## Métricas do modelo

Para demonstrar o resultado do modelo, as seguintes métricas são definidas:

- Matriz de confusão
- Cálculo da acurácia
- Total de erros e acertos
- Curva de precisão e recall
- Indíce da silhueta

Através dessas métricas, é possível avaliar a qualidade do modelo, assim como permite identificar onde ele deve melhorar.

Plotagem da matriz de confusão, além de mostrar métricas relacionadas a acurácio do modelo e quantidade total de acertos e erros.

In [ ]:
matriz_confusao = confusion_matrix(y_true, y_pred, labels=matches['first_to_score'].unique())

plt.figure(figsize=(10,7))
sns.heatmap(matriz_confusao, annot=True, fmt='d', cmap='Blues',
            xticklabels=matches['first_to_score'].unique(),
            yticklabels=matches['first_to_score'].unique())

plt.title('Matriz de Confusão - Previsão vs Realidade')
plt.xlabel('Times Previstos')
plt.ylabel('Times Reais')
plt.show()

acertos = matriz_confusao.diagonal().sum()
erros = matriz_confusao.sum() - acertos
print(f"Total de acertos: {acertos}")
print(f"Total de erros: {erros}")

acuracia = accuracy_score(y_true, y_pred)
print(f"Acurácia do modelo: {(acuracia * 100):.2f}%")


Plota um Gráfico que visa demonstra a curva de precisão e recall

In [ ]:
classes = matches['first_to_score'].unique()
y_true_bin = label_binarize(y_true, classes=classes)
y_pred_bin = label_binarize(y_pred, classes=classes)

precision = dict()
recall = dict()
for i in range(len(classes)):
    precision[i], recall[i], _ = precision_recall_curve(y_true_bin[:, i], y_pred_bin[:, i])

plt.figure(figsize=(8, 6))
for i in range(len(classes)):
    plt.plot(recall[i], precision[i], marker='.', label=classes[i])

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Curva de Precisão-Recall para cada time')
#plt.legend()
plt.grid(True)
plt.show()

In [ ]:
inertia = kmeans.inertia_
print(f"Inércia: {inertia}")

Gráfico representando o Índice da Silhueta e o Método do Cotovelo.

In [ ]:
silhouette_avg = silhouette_score(teams, kmeans.labels_)
print(f"Índice de Silhueta: {silhouette_avg}")

inertia_values = []

for n_clusters in range(1, 11):
    kmeans = KMeans(n_clusters=n_clusters)
    kmeans.fit(teams)
    inertia_values.append(kmeans.inertia_)

plt.plot(range(1, 11), inertia_values, marker='o')
plt.xlabel('Número de Clusters')
plt.ylabel('Inércia')
plt.title('Método do Cotovelo')
plt.show()